In [1]:
# main.py — train per target×horizon on trainset.csv, then validate (test + external) via validation.py
import os
import sys
import json
import subprocess
from datetime import datetime
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import gc
import tensorflow as tf

from ann import ann_model

# --- Config ---
targets = ['wind','hydro','solar','load','price']

HORIZONS = {
    "immediate": 1,      # direct next step
    "short": 6,          # 6 steps ahead
    "medium": 24,        # 24 steps ahead
    "long": 144,         # 144 steps ahead
    "strategic": 1008    # 1008 steps ahead
}

months_filter = list(range(1, 13))
# IMPROVED: Increased look_back from 6 to 24 (4 hours of history instead of 1 hour)
look_back = 24

train_ratio = 0.70
val_ratio   = 0.15
test_ratio  = 0.15

epochs   = 200
batch_size = 64
shuffle  = True
patience = 12

TRAIN_PATH = "trainset.csv"
EXT_VALIDATION_PATH = "validationset.csv"

# --- IO prep ---
for d in ["metadata", "models", "scalers", "feature_matrices", "datasets", "history", "seeds", "forecast_results"]:
    os.makedirs(d, exist_ok=True)

# Load training data (for month filter)
df_train = pd.read_csv(TRAIN_PATH)
df_train['Date']  = pd.to_datetime(df_train['timestamp'])
df_train['Year']  = df_train['Date'].dt.year
df_train['Month'] = df_train['Date'].dt.month

def read_metrics_json(path):
    with open(path, "r") as f:
        return json.load(f)

rows = []

with tqdm(total=len(targets), desc="Targets", unit="target") as tbar:
    for target in targets:
        with tqdm(total=len(HORIZONS), desc=f"{target} horizons", unit="hz", leave=False) as hbar:
            for hlabel, hsteps in HORIZONS.items():
                tqdm.write(f"\n📌 Target: {target} | Horizon: {hlabel} ({hsteps} steps)")
                # Prepare training view
                new_data = df_train[['Month', 'Year', 'Date', target]].copy()
                cap = float(new_data[target].max())

                # 1) Train per horizon
                out = ann_model(
                    new_data=new_data,
                    i=months_filter,
                    look_back=look_back,
                    cap=cap,
                    target=target,
                    train_ratio=train_ratio,
                    val_ratio=val_ratio,
                    test_ratio=test_ratio,
                    epochs=epochs,
                    batch_size=batch_size,
                    shuffle=shuffle,
                    patience=patience,
                    horizon_steps=hsteps,
                    horizon_label=hlabel
                )

                # Memory cleanup - models/scalers are already saved to disk
                tf.keras.backend.clear_session()
                gc.collect()

                # Read performance from metadata
                metadata_path = out.get('metadata_path') or os.path.join("metadata", f"{target}_{hlabel}_metadata.json")
                with open(metadata_path, "r") as f:
                    md = json.load(f)
                perf = md['performance']
                train_perf = perf['train']

                # 2) Run validation.py for internal unseen test
                env = os.environ.copy()
                env["VAL_TARGET"] = target
                env["VAL_HORIZON_LABEL"] = hlabel
                env["VAL_HORIZON_STEPS"] = str(hsteps)
                env["VAL_MONTHS"] = ",".join(map(str, months_filter))

                env["VAL_SPLIT"] = "test"
                res = subprocess.run([sys.executable, "validation.py"], env=env, capture_output=True, text=True, encoding='utf-8', errors='replace')
                if res.returncode != 0:
                    print(res.stdout); print(res.stderr)
                    raise RuntimeError(f"validation.py failed (split=test, target={target}, horizon={hlabel})")
                test_metrics_json = os.path.join("forecast_results", f"{target}_{hlabel}_test_metrics.json")
                test_metrics = read_metrics_json(test_metrics_json)

                # 3) Run validation.py for external unseen dataset
                env["VAL_SPLIT"] = "external"
                env["VAL_EXTERNAL_PATH"] = os.path.abspath(EXT_VALIDATION_PATH)
                res = subprocess.run([sys.executable, "validation.py"], env=env, capture_output=True, text=True, encoding='utf-8', errors='replace')
                if res.returncode != 0:
                    print(res.stdout); print(res.stderr)
                    raise RuntimeError(f"validation.py failed (split=external, target={target}, horizon={hlabel})")
                ext_metrics_json = os.path.join("forecast_results", f"{target}_{hlabel}_external_metrics.json")
                ext_metrics = read_metrics_json(ext_metrics_json)

                # Additional memory cleanup after subprocess calls
                gc.collect()

                # 4) Collect one row per target×horizon
                rows.append({
                    'target': target,
                    'horizon_label': hlabel,
                    'horizon_steps': hsteps,

                    # training metrics (capacity & std mape if present)
                    'train_MAPE': float(train_perf['mape']),
                    'train_STD_MAPE': float(train_perf.get('std_mape', np.nan)),
                    'train_RMSE': float(train_perf['rmse']),
                    'train_MAE' : float(train_perf['mae']),

                    # internal test (unseen part of trainset) from validation.py
                    'unseen_train_MAPE': float(test_metrics['cap_mape']),
                    'unseen_train_STD_MAPE': float(test_metrics.get('std_mape', np.nan)),
                    'unseen_train_RMSE': float(test_metrics['rmse']),
                    'unseen_train_MAE' : float(test_metrics['mae']),

                    # external unseen dataset from validation.py
                    'unseen_external_MAPE': float(ext_metrics['cap_mape']),
                    'unseen_external_STD_MAPE': float(ext_metrics.get('std_mape', np.nan)),
                    'unseen_external_RMSE': float(ext_metrics['rmse']),
                    'unseen_external_MAE' : float(ext_metrics['mae']),

                    # paths for traceability
                    'model_path'     : md.get('model_path'),
                    'model_path_best': md.get('model_path_best'),
                    'metadata_path'  : metadata_path,
                    'history_path'   : md.get('history_path'),
                })
                hbar.update(1)
        tbar.update(1)

# Save combined metrics CSV
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
results_df = pd.DataFrame(rows)
out_csv = f"metrics_results_{timestamp}.csv"
results_df.to_csv(out_csv, index=False)
print(f"\n✅ Metrics saved to '{out_csv}'")


c:\Users\gepap\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
                                                  
Targets:   0%|          | 0/5 [00:00<?, ?target/s]  


📌 Target: wind | Horizon: immediate (1 steps)
📊 Total samples: 526,032
🕒 Horizon label: immediate | steps ahead: 1
📊 Train: 368,222 (70.0%)
📊 Validation: 78,904 (15.0%)
📊 Test: 78,906 (15.0%)
📊 Training features: (368198, 24)
📊 Validation features: (78880, 24)
📊 Test features: (78882, 24)



📊 Performance Results
🎯 Train  - MAPE: 0.282% | StdMAPE: 1198001.875% | StdMAPE(ε): 1198001.875% | RMSE: 18.598, MAE: 4.226
🔍 Val    - MAPE: 0.305% | StdMAPE: 1141733.500% | StdMAPE(ε): 1141733.500% | RMSE: 17.458, MAE: 4.577
🚀 Test   - MAPE: 0.288% | StdMAPE: 1253223.250% | StdMAPE(ε): 1253223.250% | RMSE: 20.107, MAE: 4.313
✅ Good generalization (val ≈ test).
✅ Model saved (method 1): models/wind_immediate_model.h5

✅ Model (best) saved to: models/wind_immediate_model_best.h5
✅ Model (final w/ best weights) saved to: models/wind_immediate_model.h5
✅ Scalers saved to: scalers/wind_immediate_sc_X.pkl, scalers/wind_immediate_sc_y.pkl
✅ History saved to: history/wind_immediate_history.json
✅ Metadata saved to: metadata/wind_immediate_metadata.json
✅ Seed windows saved to: seeds/wind_immediate_seed_window_raw.npy, seeds/wind_immediate_seed_window_scaled.npy
✅ Test data saved to: datasets/wind_immediate_test_data.csv




                                                  
Targets:   0%|          | 0/5 [11:04<?, ?target/s]           


📌 Target: wind | Horizon: short (6 steps)
📊 Total samples: 526,032
🕒 Horizon label: short | steps ahead: 6
📊 Train: 368,222 (70.0%)
📊 Validation: 78,904 (15.0%)
📊 Test: 78,906 (15.0%)
📊 Training features: (368192, 24)
📊 Validation features: (78874, 24)
📊 Test features: (78876, 24)


...

                                                                
Targets:  80%|████████  | 4/5 [4:49:58<1:10:57, 4257.80s/target]


📌 Target: price | Horizon: strategic (1008 steps)
📊 Total samples: 526,032
🕒 Horizon label: strategic | steps ahead: 1008
📊 Train: 368,222 (70.0%)
📊 Validation: 78,904 (15.0%)
📊 Test: 78,906 (15.0%)
📊 Training features: (367190, 24)
📊 Validation features: (77872, 24)
📊 Test features: (77874, 24)



📊 Performance Results
🎯 Train  - MAPE: 1.265% | StdMAPE: 147691.109% | StdMAPE(ε): 147691.109% | RMSE: 166.613, MAE: 88.312
🔍 Val    - MAPE: 8.001% | StdMAPE: 2698996.250% | StdMAPE(ε): 2698996.250% | RMSE: 769.259, MAE: 558.629
🚀 Test   - MAPE: 4.350% | StdMAPE: 5367725.500% | StdMAPE(ε): 5367725.500% | RMSE: 422.494, MAE: 303.704
⚠️  Possible overfitting (val >> train).
✅ Model saved (method 1): models/price_strategic_model.h5

✅ Model (best) saved to: models/price_strategic_model_best.h5
✅ Model (final w/ best weights) saved to: models/price_strategic_model.h5
✅ Scalers saved to: scalers/price_strategic_sc_X.pkl, scalers/price_strategic_sc_y.pkl
✅ History saved to: history/price_strategic_history.json
✅ Metadata saved to: metadata/price_strategic_metadata.json
✅ Seed windows saved to: seeds/price_strategic_seed_window_raw.npy, seeds/price_strategic_seed_window_scaled.npy
✅ Test data saved to: datasets/price_strategic_test_data.csv


Targets: 100%|██████████| 5/5 [4:53:48<00:00, 3525.63s/target]  


✅ Metrics saved to 'metrics_results_20251123_2313.csv'
